# 3. Interactive Debugging tool

Objective: This section of the project deals with a code implementing interactive widgets to perform the following tasks  

1. Allows the users to select a game using a slider object and print the match summary ie., team names, number of golas by each team and the winner of the match

2. Allows the user to select any game with it's ID, aceess all the goal/Shot events of that selected game to siplay the position of the punk on the rink.

3. Allows the user to input an year, toggle between a regular and a play-off season, select a home and an away country to display the match summary.

Refer to ift6758/data/question_2_4.py for functions used to import the data.

This takes a while to run, so run it only once, save output as csv and import the csv for future uses

In [2]:
import json

In [3]:
import pandas as pd
df = pd.read_csv('tidy_df.csv')

In [4]:
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

In [5]:
import importlib
import matplotlib.pyplot as plt

Code for match summary

In [6]:
df['game_id_numeric'] = pd.to_numeric(df['game_id'])
list_game_id = df['game_id_numeric'].unique().tolist()
data_folder_path = "./"

def Game_ID(val):
    match_summary =[]
    match_summary_col = ['Home Team','Home Team Score','Home Team Attempts','Away Team','Away Team Score','Away Team Attempts','Winner']
    count = 0
    with open(f'{data_folder_path}{val}.json','r') as f:
        data_Q3 = json.load(f)
        df_Q3 = pd.DataFrame(data_Q3)
        
    if df_Q3['liveData']['linescore']['hasShootout'] is not True:
        Home_Team = df_Q3['gameData']['teams']['home']['name']
        Away_Team = df_Q3['gameData']['teams']['away']['name']
        Home_Team_Score = df_Q3['liveData']['linescore']['teams']['home']['goals']
        Home_Team_Attempts = df_Q3['liveData']['linescore']['teams']['home']['shotsOnGoal']
        Away_Team_Score = df_Q3['liveData']['linescore']['teams']['away']['goals']
        Away_Team_Attempts = df_Q3['liveData']['linescore']['teams']['away']['shotsOnGoal']
        if Home_Team_Score > Away_Team_Score:
            winner = Home_Team
        else:
            winner = Away_Team
        match_summary.append([Home_Team,Home_Team_Score,Home_Team_Attempts,Away_Team,Away_Team_Score,Away_Team_Attempts,winner])
        match_summary = pd.DataFrame(match_summary, columns = match_summary_col)
        print("Game summary of ",val," between",Home_Team," and ",Away_Team)

        return(match_summary)
    

Game_ID = widgets.interact(Game_ID, val=widgets.SelectionSlider(description='Game_ID',options=list_game_id))


interactive(children=(SelectionSlider(description='Game_ID', options=(2016020001, 2016020002, 2016020003, 2016…

This portion of the code allows the users to toggle between all games (seasons 2016 thru 2020) either a regular season game or a play-off match and access all shot/goal events of the corresponding games to display the punk position on the rink. 

In [7]:
img_folder_path = "../figures/"

In [8]:
# Game id is argumented in this function

df['game_id_numeric'] = pd.to_numeric(df['game_id'])
list_game_id = df['game_id_numeric'].unique().tolist()

def Game_ID(var):
    global cnt
    global list_Event_Id
    global check, display
    check = var
    cnt = -1
    list_Event_Id = []
    with open(f'{data_folder_path}{var}.json','r') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        for i in df['liveData']['plays']['allPlays']:
                if i['result']['event'] == 'Shot' or i['result']['event'] == 'Goal':
                    cnt += 1
                    list_Event_Id.append(i['about']['eventIdx']) 
x = widgets.interact(Game_ID, var=widgets.SelectionSlider(description='Game_ID',options=list_game_id))


interactive(children=(SelectionSlider(description='Game_ID', options=(2016020001, 2016020002, 2016020003, 2016…

In [15]:
#Event id (Shot/Goal) is argumented for a selected game to locate punk on rink 

def Event_ID(val):
    global temp
    temp = val
    df_group = df.loc[df['game_id'] == check]
    df_group = df_group.loc[df['event_id'] == temp]
    player=df_group['attacking_player'].tolist()
    team=df_group['attacking_team'].tolist()
    period=df_group['period'].tolist()
    type=df_group['shot_type'].tolist()
    
    img = plt.imread(f'{img_folder_path}nhl_rink.png') 
    fig = plt.figure(figsize=(35,15))
    fig.suptitle(f'Game {check}:{type[0]} by {player[0]} of {team[0]} in period {period[0]}',fontsize=25,y=.95)
    ax1 = fig.add_subplot(1,1,1)
    ax1.tick_params(labelsize=25)
    ax1.imshow(img,extent=[-100,100,-42.5,42.5])
    ax1.plot(df_group['x_coordinates'],df_group['y_coordinates'],'b.',markersize=50)
    ax1.set_title("Shot Locator",fontsize=30)
    ax1.set_xlim(-100,100)
    ax1.set_ylim(-42.5,42.5)
    
y = widgets.interact(Event_ID, val=widgets.SelectionSlider(description='Event_ID',options=list_Event_Id))

interactive(children=(SelectionSlider(description='Event_ID', options=(7, 13, 24, 26, 47, 52, 54, 56, 62, 66, …

In [10]:
#Game year selection

year = widgets.IntSlider(
    value=2017,
    min=2016,
    max=2020,
    step=1,
    description='Year',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(year)


# Switch between regular season and play-offs

game_type = widgets.RadioButtons(
    options=['Regular', 'Playoffs'],
    description='Season:',
    disabled=False
)
display(game_type)

IntSlider(value=2017, continuous_update=False, description='Year', max=2020, min=2016)

RadioButtons(description='Season:', options=('Regular', 'Playoffs'), value='Regular')

In [11]:
# List teams based on a regular season and play-off season

if game_type.value=="Regular":
    k = "R"
elif game_type.value=="Playoffs":
    k = "P"

df_group1 = df.groupby('season')
df_group2 = df_group1.get_group((year.value*10000+(year.value+1)))
df_group3 = df_group2.groupby('season_type')
df_group4 = df_group3.get_group(k)

home_team_list_R = df_group2['home_team'].unique().tolist()
away_team_list_R = df_group2['away_team'].unique().tolist()

home_team_list_P = df_group4['home_team'].unique().tolist()
away_team_list_P = df_group4['away_team'].unique().tolist()

if game_type.value=="Regular":
    home_team_list = home_team_list_R
    away_team_list = away_team_list_R
elif game_type.value=="Playoffs":
    home_team_list = home_team_list_P
    away_team_list = away_team_list_P

# Function to print game summary

def match_summary(games_av):
    match_summary =[]
    match_summary_col = ['Home Team','Home Team Score','Home Team Attempts','Away Team','Away Team Score','Away Team Attempts','Winner']
    count = 0
    with open(f'{data_folder_path}{games_av.value}.json','r') as f:
        data_Q3 = json.load(f)
        df_Q3 = pd.DataFrame(data_Q3)
        if df_Q3['liveData']['linescore']['hasShootout'] is not True:
            Home_Team = df_Q3['gameData']['teams']['home']['name']
            Away_Team = df_Q3['gameData']['teams']['away']['name']
            Home_Team_Score = df_Q3['liveData']['linescore']['teams']['home']['goals']
            Home_Team_Attempts = df_Q3['liveData']['linescore']['teams']['home']['shotsOnGoal']
            Away_Team_Score = df_Q3['liveData']['linescore']['teams']['away']['goals']
            Away_Team_Attempts = df_Q3['liveData']['linescore']['teams']['away']['shotsOnGoal']
            if Home_Team_Score > Away_Team_Score:
                winner = Home_Team
            else:
                winner = Away_Team
            match_summary.append([Home_Team,Home_Team_Score,Home_Team_Attempts,Away_Team,Away_Team_Score,Away_Team_Attempts,winner])
            

            match_summary = pd.DataFrame(match_summary, columns = match_summary_col)

    print(match_summary)
    


In [12]:
# Home team selection from pull-dowm menu

Home_team = widgets.Dropdown(
    description='Home Team:   ',
    value=home_team_list[0],
    options=home_team_list

)

# Away team selection from pull-dowm menu

Away_team = widgets.Dropdown(
    description='Away Team:   ',
    value=away_team_list[0],
    options=away_team_list

)

container2 = widgets.HBox(children=[Home_team, Away_team])
container2

In [13]:
def validate():
    if Home_team.value in df['home_team'].unique() and Away_team.value in df['away_team'].unique():
        if Home_team.value != Away_team.value:
            return True      
    else:
        return False
    
# Function that filters the games played between a selected home team and an away team

def games_list():
    if validate():
        global cnt
        match_list = []
        cnt = 0
        df_group1 = df.groupby('season')
        df_group2 = df_group1.get_group((year.value*10000+(year.value+1)))
        df_group3 = df_group2.groupby('season_type')
        df_group4 = df_group3.get_group(k)
        
        for index, row in df_group4.iterrows():
            if df_group4['home_team'][index] == Home_team.value and df_group4['away_team'][index] == Away_team.value:
                cnt += 1
                match_list.append(df_group4['game_id'][index])
                
        match_list_all = match_list               # to find unique game ids
        match_list_temp = []
        for i in match_list_all:
            if i not in match_list_temp:
                match_list_temp.append(i)
                        
        if cnt == 0:
            if k == "P":
                print(f'No Play-off games played between {Home_team.value} and {Away_team.value} in {(year.value*10000+(year.value+1))} season')
            elif k == "R":
                print("Error in Code")
        elif cnt > 0:
            games_list = match_list_temp
            
            
            selected_game = widgets.Dropdown(
                description='Avaibale Games:   ',
                value=match_list_temp[0],
            options=match_list_temp
            )
            display(selected_game)
            
            
            return selected_game
        
games_av = games_list()

Dropdown(description='Avaibale Games:   ', options=(2017020001,), value=2017020001)

In [14]:
if cnt > 0:
    match_summary(games_av)

       Home Team  Home Team Score  Home Team Attempts            Away Team  \
0  Winnipeg Jets                2                  37  Toronto Maple Leafs   

   Away Team Score  Away Team Attempts               Winner  
0                7                  31  Toronto Maple Leafs  
